In [2]:
# Initial imports
import datetime as dt
import hvplot.pandas
import json
import math
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import requests

%matplotlib inline

In [3]:
coinbase_api_url = "https://api.pro.coinbase.com"
products_api_url = f"{coinbase_api_url}/products"
currencies_api_url = f"{coinbase_api_url}/currencies"

In [4]:
currencies_response = requests.get(currencies_api_url)
currencies = currencies_response.json()
currencies_df = pd.DataFrame(currencies)
currencies_df = pd.concat([currencies_df, pd.io.json.json_normalize(currencies_df["details"])], axis=1).drop(columns=["details"])
currencies_df.head()

,convertible_to,id,max_precision,message,min_size,name,status,crypto_address_link,crypto_transaction_link,display_name,group_types,max_withdrawal_amount,min_withdrawal_amount,network_confirmations,processing_time_seconds,push_payment_methods,sort_order,symbol,type
0,[],ALGO,0.000001,,1,Algorand,online,https://algoexplorer.io/address/{{address}},https://algoexplorer.io/tx/{{txId}},,[],1000000.0,0.1000,1,5,[crypto],93,A,crypto
1,[],DASH,0.00000001,,1,Dash,online,https://chain.so/address/DASH/{{address}},https://chain.so/tx/DASH/{{address}},,[],10000.0,0.0001,2,0,[crypto],47,,crypto
2,[],OXT,0.00000001,,1,Orchid,online,https://etherscan.io/token/0x4575f41308EC1483f...,https://etherscan.io/tx/0x{{txId}},,[],1000000.0,0.0100,35,0,[crypto],48,,crypto
3,[],ATOM,0.000001,,1,Cosmos,online,https://cosmos.bigdipper.live/account/{{address}},https://cosmos.bigdipper.live/transactions/{{t...,,[],100000.0,0.1000,0,5,[crypto],51,,crypto
4,[],KNC,0.00000001,,1,Kyber Network,online,https://etherscan.io/token/0xdd974d5c2e2928dea...,https://etherscan.io/tx/0x{{txId}},,[],2156253.0,0.0100,35,0,[crypto],120,Ξ,crypto


In [5]:
products_response = requests.get(products_api_url)
products = products_response.json()
products_df = pd.DataFrame(products)
products_df.head()

,base_currency,base_increment,base_max_size,base_min_size,cancel_only,display_name,id,limit_only,margin_enabled,max_market_funds,min_market_funds,post_only,quote_currency,quote_increment,status,status_message,trading_disabled
0,BCH,0.00000001,100.00000000,0.01000000,False,BCH/EUR,BCH-EUR,False,False,300000,10,False,EUR,0.01000000,online,,False
1,OMG,0.10000000,500000.00000000,1.00000000,False,OMG/EUR,OMG-EUR,False,False,100000,1,False,EUR,0.00010000,online,,False
2,LINK,0.01000000,90000.00000000,1.00000000,False,LINK/GBP,LINK-GBP,False,False,100000,10,False,GBP,0.00001000,online,,False
3,AAVE,0.00100000,1200.00000000,0.01000000,False,AAVE/BTC,AAVE-BTC,False,False,5.2,0.0001,False,BTC,0.00000001,online,,False
4,LINK,0.01000000,6500.00000000,0.10000000,False,LINK/BTC,LINK-BTC,False,False,6.0,0.0001,False,BTC,0.00000001,online,,False


In [7]:
def get_candle_data(id, start_date):
    start_str = dt.datetime.strftime(start_date, "%Y-%m-%d")
    end_str = dt.datetime.strftime(start_date + dt.timedelta(days=300), "%Y-%m-%d")
    candles_url = f"{products_api_url}/{id}/candles?start={start_str}&end={end_str}&granularity=86400"
    candles_response = requests.get(candles_url)
    candles = candles_response.json()
    candles_df = pd.DataFrame(candles, columns=["time", "low", "high", "open", "close", "volume"])
    return candles_df

In [28]:
fiat_currencies = currencies_df[currencies_df["type"] == "fiat"]["id"].tolist()
fiat_products_df = products_df[products_df["quote_currency"].isin(fiat_currencies)]

today = dt.datetime.today()
base = dt.datetime.strptime("2009-01-01", "%Y-%m-%d")
delta_days = (today - base).days
iterations = math.floor(delta_days / 300) + 1
date_list = [base + dt.timedelta(days=x*300) for x in range(iterations)]

data_df = pd.DataFrame()
for idx, row in fiat_products_df.iterrows():
    product_id = row["id"]
    base_currency = row["base_currency"]
    quote_currency = row["quote_currency"]
    for start_date in date_list:
        df = get_candle_data(product_id, start_date)
        if not df.empty:
            df["base_currency"] = base_currency
            df["quote_currency"] = quote_currency
            data_df = data_df.append(df)

In [29]:
# convert epoch time to date
data_df = data_df.drop_duplicates()
data_df["date"] = pd.to_datetime(data_df["time"], unit="s")
data_df = data_df.drop(columns=["time"])

In [30]:
# visualize volume by fiat currency
fiat_volume_df = data_df[["date", "quote_currency", "volume"]].groupby(["date", "quote_currency"]).sum()
fiat_volume_df.hvplot.line(x="date", y="volume", by="quote_currency", title="Volume of Trading Activity by Fiat Currency", height=300, width=1000)

:NdOverlay   [quote_currency]
   :Curve   [date]   (volume)

In [31]:
# visualize USD volume by base currency
usd_volume_df = data_df[data_df.quote_currency == "USD"].groupby(["date", "base_currency"])["volume"].sum()
usd_volume_df.hvplot.line(x="date", y="volume", by="base_currency", title="USD Volume of Trading Activity", height=500, width=1000)

:NdOverlay   [base_currency]
   :Curve   [date]   (volume)